# GPIO Demo
This demo shows how to use the Xilinx's MMIO driver to interact with partially reconfigurable regions.

## Download the static bitstream
We first need to download the static or full bitstream before any partial bitstreams can be downloaded. Note that if the bitstream is not in the same directory as the notebook then the full path needs to be provided.


In [1]:
from pynq import Overlay

FULL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio/"
PARTIAL_BITSTREAM_PATH = "/usr/local/lib/python3.6/dist-packages/prio/partial/"

overlay = Overlay(FULL_BITSTREAM_PATH + "prio.bit")

## Set up the reconfigurable region
Notice that as with the full bitstream, the full path to the partial bitstream must be provided when is located outside of the current notebook's directory.

We will download partial bitstream and retrieve the initilized mmio driver for future use. Note that the pynq package provides a wrapper for AXI GPIO modules specifically, but we will use the MMIO driver instead to demonstrate how to build custom drivers.


In [2]:
overlay.pr_download("pr_0", PARTIAL_BITSTREAM_PATH + "pr_0_gpio.bit")
gpio0 = overlay.pr_0.axi_gpio_0.mmio

## MMIO Demos
The following Demos use MMIO to read and write to the registers in the GPIO hardware inside different PR regions to demonstrate simple examples increasing in complexity. 

## Demo: Constant Output
Outputs a constant pattern to the **`pr_0`** pins.

** Hardware Setup:** For this demo, we suggest using at least one LED to check the output of every pin. These LEDs should be hooked up to arduino pins 0-7.

In [3]:
# Constants for register offsets
DATA_OFFSET = 	0x00
TRI_OFFSET  =	0x04

# Convenient constants for setting the tri-state buffers
ALL_INPUTS  = 	0xFFFF
ALL_OUTPUTS =   0x00

# Desired pattern of gpio output.
LED_PATTERN = 0b10101010

# Sets the direction of all pins in region 0 to output.
gpio0.write(TRI_OFFSET, ALL_OUTPUTS)

#sets the value of all the pins in region 0 so everyother pin is high.
gpio0.write(DATA_OFFSET, LED_PATTERN)

## Demo: Binary Counter
This demo will output the count, in binary, of a counter to the pins of **`pr_0`**.

** Hardware Setup:** This Demo should use the same LED setup as the **`Constant Output`** Demo

In [4]:
import time

OUTPUTS_OFF = 0b0

# Sets the direction of all pins in region 0 to  output.
gpio0.write(TRI_OFFSET, ALL_OUTPUTS)

# Counts from 0 to 15 in binary
for output in range(0, 16):
    
    gpio0.write(DATA_OFFSET,output)
    
    #pauses for a half second
    time.sleep(0.5)
    
#sets the value of all the pins in region 0 so everyother pin is high.
gpio0.write(DATA_OFFSET, OUTPUTS_OFF)

## Demo: Blink Test
The following demonstrates the Input and Output functionalities of GPIO interface by toggling the value on Arduino pin 8, and reading that value from Arduino pin 34. This value will then be echoed to Arduino pin 0.

** Hardware Setup:** Keep an LED on Arduino pin 0. Connect a wire from Arduino pin 8 to Arduino pin 34.

The LED on Arduino pin 0 and the cell will indicate the current value being read from Arduino pin 34.

### Setup additional reconfigurable regions
Since this demo requires two additional PR regions, we must set them up with the following code.

In [5]:
overlay.pr_download("pr_1", PARTIAL_BITSTREAM_PATH + "pr_1_gpio.bit")
gpio1 = overlay.pr_1.axi_gpio_0.mmio

overlay.pr_download("pr_3", PARTIAL_BITSTREAM_PATH + "pr_3_gpio.bit")
gpio3 = overlay.pr_3.axi_gpio_0.mmio

In [6]:
import time

# Since only the right most pin is needed as an output
# for the LEDs, that one is set to an output(0), while
# the rest are set to inputs (1).
gpio0.write(TRI_OFFSET, 0xe)

gpio1.write(TRI_OFFSET, ALL_OUTPUTS)
gpio3.write(TRI_OFFSET, ALL_INPUTS)

led_value = 0

for i in range(20):
    time.sleep(0.5)
    gpio1.write(DATA_OFFSET,led_value)
    led_value = led_value ^ 0b10000001
    
    val = gpio3.read(DATA_OFFSET) & 0x01
    
    gpio0.write(DATA_OFFSET, val)
    if val:
        print('on')
    else:
        print('off')
    

off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on
off
on


## The Use of a Wrapper Class
MMIO can be far easier to use when its wrapped inside of a class. Below is a simple class that uses the MMIO driver from the partial regions to set the tri-state buffer and read and write values.

In [7]:
class GPIO():	
    # Constants for register offsets
	DATA_OFFSET = 	0x00
	TRI_OFFSET =	0x04
	 
    # Convenient constants for setting the direction of tri-state buffers
	ALL_INPUTS = 	0xFFFF
	ALL_OUTPUTS = 	0x0000

    
    # Extracts the mmio driver from the PR region assuming
    # that Xilinx's driver associated with the AXI interface
    # is named "axi_gpio_0"
	def __init__(self, partial_region):
		self._mmio = partial_region.axi_gpio_0.mmio

    # Used to set the directions of the tri-state buffers
	def setDirection(self, direction):
		self._mmio.write(self.TRI_OFFSET, direction)

    # Used to retrieve the directions of the tri-state buffers
	def getDirection(self):
		return self._mmio.read(self.TRI_OFFSET)

    # Used to set the value of pins when the tri-state buffers are
    # set as outputs
	def setValue(self, value):
		self._mmio.write(self.DATA_OFFSET, value)
        
    # Used to retrieve the value of pins when the tri-state buffers are
    # set as inputs
	def getValue(self):
		return self._mmio.read(self.DATA_OFFSET)

## GPIO Wrapper Demos
Below are all of the demos repeated with the GPIO wrapper class. There are a few extra peculiarities that should be noted.

## Demo: Constant Output
Outputs a constant pattern to the **`pr_0`** pins.

** Hardware Setup:** For this demo, we suggest using at least one LED to check the output of every pin. These LEDs should be hooked up to arduino pins 0-7.

In [8]:
overlay.pr_download("pr_0", PARTIAL_BITSTREAM_PATH + "pr_0_gpio.bit")

gpio0 = GPIO(overlay.pr_0)

In [9]:
LED_PATTERN = 0b1010

gpio0.setDirection(ALL_OUTPUTS)

gpio0.setValue(LED_PATTERN)

## Demo: Binary Counter
This demo will output the count, in binary, of a counter to the pins of **`pr_0`**.

** Hardware Setup:** This Demo should use the same LED setup as the **`Constant Output`** Demo

In [10]:
import time

gpio0.setDirection(ALL_OUTPUTS)

for output in range(0, 16):
    
    gpio0.setValue(output)
    
    time.sleep(0.5)

## Demo: Blink Test
The following demonstrates the Input and Output functionalities of GPIO interface by toggling the value on Arduino pin 8, and reading that value from Arduino pin 34. This value will then be echoed to Arduino pin 0.

** Hardware Setup:** Keep an LED on Arduino pin 0. Connect a wire from Arduino pin 8 to Arduino pin 34.


The LED on Arduino pin 0 and the cell will indicate the current value being read from Arduino pin 34.

### Setup additional reconfigurable regions
Since this demo requires two additional PR regions, we must set them up with the following code.

**IMPORTANT:** It's important to note below that after a partial region is set up, and a bitstream for that region is downloaded, the GPIO driver must be instantiated before another partial region is set up. Once another partial region is setup, the previous one will lose it's GPIO driver, and will have a generic MMIO axi interface driver. 

In [11]:
overlay.pr_download("pr_1", PARTIAL_BITSTREAM_PATH + "pr_1_gpio.bit")
gpio1 = GPIO(overlay.pr_1)

overlay.pr_download("pr_3", PARTIAL_BITSTREAM_PATH + "pr_3_gpio.bit")
gpio3 = GPIO(overlay.pr_3)

In [12]:
import time

gpio0.setDirection(0xe)

gpio1.setDirection(ALL_OUTPUTS)
gpio3.setDirection(ALL_INPUTS)

led_value = 0

for i in range(10):
    time.sleep(0.5)
    gpio1.setValue(led_value)
    led_value = led_value ^ 0b10000001
    
    val = gpio3.getValue() & 0x01
    
    gpio0.setValue(val)
    print(val)
    if val:
        print('on')
    else:
        print('off')
    

0
off
1
on
0
off
1
on
0
off
1
on
0
off
1
on
0
off
1
on
